In [1]:
import pandas as pd 
df = pd.read_csv("./cleaned_data.csv")

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (2,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.drop("Unnamed: 0", inplace = True, axis =1 )

In [3]:
df.head()

,Goal,Target,Indicator,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,SeriesCode,SeriesDescription,Source,Nature,Reporting Type,Sex,Units
0,4,4.1,4.1.1,4,Afghanistan,2013,11.0,2013,25.0,16.0,168.0,0.0,0.0,0.0,1.0
1,4,4.1,4.1.1,4,Afghanistan,2013,13.0,2013,25.0,16.0,167.0,0.0,0.0,0.0,1.0
2,4,4.1,4.1.1,4,Afghanistan,2016,21.5,2016,25.0,16.0,166.0,0.0,0.0,2.0,1.0
3,4,4.1,4.1.1,4,Afghanistan,2016,22.5,2016,25.0,16.0,166.0,0.0,0.0,1.0,1.0
4,4,4.1,4.1.1,4,Afghanistan,2016,22.0,2016,25.0,16.0,166.0,0.0,0.0,0.0,1.0


With the data cleaned we can address the questions brought forward. We can forecast for 2019.

In [4]:
df["Time_Detail"].value_counts()

2006         12809
2014         11942
2016         11026
2011         10866
2012         10821
2015          9925
2013          9345
2017          8453
2009          8429
2018          8354
2005          8164
2010          7760
2007          7384
2003          6466
2008          6369
2004          6115
2000          5941
2001          4859
2002          3810
2014          3404
2016          2916
2017          2548
2015          2349
2006          2288
2010          2208
2011          2121
2013          1915
2012          1603
2005          1546
2018          1472
2009          1217
2019          1146
2004          1033
2000           934
2008           924
2007           902
2003           757
2002           670
2001           668
2019           156
2017-18         36
2018-19         36
2018-2019       18
2017-2018       12
2016-2017       12
2012-2020       11
2011-2012        9
2014-2015        6
2019-2020        4
2013-2014        3
Name: Time_Detail, dtype: int64

Not every bin is refined for a particular year. More so, muliple bins exist for the sam year. This needs to be fixed 

In [5]:
df["Time_Detail"] = df["Time_Detail"].astype(str)

In [6]:
df["Time_Detail"].value_counts()

2014         15346
2006         15097
2016         13942
2011         12987
2012         12424
2015         12274
2013         11260
2017         11001
2010          9968
2018          9826
2005          9710
2009          9646
2007          8286
2008          7293
2003          7223
2004          7148
2000          6875
2001          5527
2002          4480
2019          1302
2017-18         36
2018-19         36
2018-2019       18
2017-2018       12
2016-2017       12
2012-2020       11
2011-2012        9
2014-2015        6
2019-2020        4
2013-2014        3
Name: Time_Detail, dtype: int64

In [7]:
df["Time_Detail"].value_counts().index # We can just drop the instances that span two years. 

Index(['2014', '2006', '2016', '2011', '2012', '2015', '2013', '2017', '2010',
       '2018', '2005', '2009', '2007', '2008', '2003', '2004', '2000', '2001',
       '2002', '2019', '2017-18', '2018-19', '2018-2019', '2017-2018',
       '2016-2017', '2012-2020', '2011-2012', '2014-2015', '2019-2020',
       '2013-2014'],
      dtype='object')

In [8]:
mulityears = ['2018-19', '2017-18', '2018-2019', '2017-2018','2016-2017', '2012-2020', '2011-2012', '2014-2015', '2019-2020','2013-2014']

In [9]:
df.loc[df["Time_Detail"] == '2012-2020'].index

Int64Index([80340, 80341, 80342, 80349, 80380, 80381, 80382, 80398, 80399,
            80400, 80543],
           dtype='int64')

In [10]:
for years in mulityears:
    df.drop(df.loc[df["Time_Detail"] == years].index , axis = 0,inplace = True)

#df["Time_Detail"].drop(['2012-2020', '2011-2012', '2014-2015', '2019-2020', '2013-2014'] ,axis = 0 , inplace = True)

In [11]:
df["Time_Detail"].value_counts()

2014    15346
2006    15097
2016    13942
2011    12987
2012    12424
2015    12274
2013    11260
2017    11001
2010     9968
2018     9826
2005     9710
2009     9646
2007     8286
2008     7293
2003     7223
2004     7148
2000     6875
2001     5527
2002     4480
2019     1302
Name: Time_Detail, dtype: int64

With that out of the way we can explore the data and model. 

We must select a Response variable. Let's select value. 

Value is a continous numeric value. Since are trying to predict a value it is best to use a supervised learning method in predictions. Right now it is a great idea to start with a simple linear regression model and increase complexity 

Lets Split the dataset into labels and data

The index shouldn't be shuffled yet, we must first take out our testing and training data.

In [12]:
X = df.drop("Value", axis = 1)
y = df["Value"]

In [13]:
y_test = y.loc[X.loc[df["Time_Detail"] == '2019'].index]
y_train =  y.drop(X.loc[df["Time_Detail"] == '2019'].index )

In [14]:
X_test = X.loc[ X.loc[df["Time_Detail"] == '2019'].index ]
X_train = X.loc[ X.loc[df["Time_Detail"] != '2019'].index ]

In [15]:
sum(X_test.index == y_test.index) / len(y_test) # Wooo! b

1.0

With the data split we are ready for some exploration. 

# Linear Exploration 

In [16]:
X_train.corr()

,Goal,GeoAreaCode,TimePeriod,SeriesCode,SeriesDescription,Source,Nature,Reporting Type,Sex,Units
Goal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GeoAreaCode,NaN,1.000000,0.014566,0.001126,-0.022776,-0.028812,0.005651,NaN,-0.000887,0.007374
TimePeriod,NaN,0.014566,1.000000,-0.102750,0.135094,-0.139362,-0.120190,NaN,-0.009778,-0.045911
SeriesCode,NaN,0.001126,-0.102750,1.000000,-0.085385,0.117533,0.043763,NaN,0.276414,-0.468219
SeriesDescription,NaN,-0.022776,0.135094,-0.085385,1.000000,-0.303680,-0.836120,NaN,-0.089345,-0.477208
Source,NaN,-0.028812,-0.139362,0.117533,-0.303680,1.000000,0.419162,NaN,0.007344,0.051302
Nature,NaN,0.005651,-0.120190,0.043763,-0.836120,0.419162,1.000000,NaN,0.137945,0.218194
Reporting Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,NaN,-0.000887,-0.009778,0.276414,-0.089345,0.007344,0.137945,NaN,1.000000,-0.187520
Units,NaN,0.007374,-0.045911,-0.468219,-0.477208,0.051302,0.218194,NaN,-0.187520,1.000000


Not many things have a high linear correlation. Where this is a correlation higher than 10% it appears to be on ordinal variables that describe our seriesCode. 

The series code is intended to tell us which indicator is being measured. 

Our data set isn't the largest and has a decent amount of features, because of this I think we should use a regularized linear model. Lasso regression uses the $L_1$ norm when optimizing our objective function, thus I will use it as it will automatically weight useless variables to zero. 

Lasso regression is doesn't do well when features represent the same thing so I will look at our VIF function to measure similarity. 

In [17]:
df.head()

,Goal,Target,Indicator,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,SeriesCode,SeriesDescription,Source,Nature,Reporting Type,Sex,Units
0,4,4.1,4.1.1,4,Afghanistan,2013,11.0,2013,25.0,16.0,168.0,0.0,0.0,0.0,1.0
1,4,4.1,4.1.1,4,Afghanistan,2013,13.0,2013,25.0,16.0,167.0,0.0,0.0,0.0,1.0
2,4,4.1,4.1.1,4,Afghanistan,2016,21.5,2016,25.0,16.0,166.0,0.0,0.0,2.0,1.0
3,4,4.1,4.1.1,4,Afghanistan,2016,22.5,2016,25.0,16.0,166.0,0.0,0.0,1.0,1.0
4,4,4.1,4.1.1,4,Afghanistan,2016,22.0,2016,25.0,16.0,166.0,0.0,0.0,0.0,1.0


In [18]:
df.columns

Index(['Goal', 'Target', 'Indicator', 'GeoAreaCode', 'GeoAreaName',
       'TimePeriod', 'Value', 'Time_Detail', 'SeriesCode', 'SeriesDescription',
       'Source', 'Nature', 'Reporting Type', 'Sex', 'Units'],
      dtype='object')

In [19]:
numerical = df.drop(['Goal', 'Target', 'Indicator', "GeoAreaName","TimePeriod", "SeriesDescription", "Value", "Units"] ,axis =1 )

In [20]:
numerical = numerical.astype(float)

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
#Create a df to conviently display the info
def vif(data):
    vif_data = pd.DataFrame()
    vif_data["feature"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values , i)for i in range(len(data.columns))]
    print(vif_data)
vif(numerical)

          feature        VIF
0     GeoAreaCode   3.514346
1     Time_Detail  53.140091
2      SeriesCode   7.153267
3          Source  57.595752
4          Nature   4.802639
5  Reporting Type        NaN
6             Sex   1.940683


/opt/tljh/user/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


Anything over 10 has a high colinearity with the other variables. I will drop the 2 highest scores and the Nan. 

In [22]:
numerical.drop(["Time_Detail", "Source",  "Reporting Type"], inplace =True, axis =1 )

In [23]:
vif(numerical)

       feature       VIF
0  GeoAreaCode  2.741602
1   SeriesCode  4.147030
2       Nature  3.101338
3          Sex  1.921481


Like that our varibales no longer describe each other at a high degree. 

Sinc we intend to use a regularized model we must scale the features, as regularized models are sensitive to scales. 

In [24]:
def modeling_features(df):
    return df.drop(["SeriesDescription","Time_Detail", "Source", "Reporting Type", 'Goal', 'Target', 'Indicator', "GeoAreaName","TimePeriod", "Units"], axis = 1)   

In [25]:
X_train

,Goal,Target,Indicator,GeoAreaCode,GeoAreaName,TimePeriod,Time_Detail,SeriesCode,SeriesDescription,Source,Nature,Reporting Type,Sex,Units
0,4,4.1,4.1.1,4,Afghanistan,2013,2013,25.0,16.0,168.0,0.0,0.0,0.0,1.0
1,4,4.1,4.1.1,4,Afghanistan,2013,2013,25.0,16.0,167.0,0.0,0.0,0.0,1.0
2,4,4.1,4.1.1,4,Afghanistan,2016,2016,25.0,16.0,166.0,0.0,0.0,2.0,1.0
3,4,4.1,4.1.1,4,Afghanistan,2016,2016,25.0,16.0,166.0,0.0,0.0,1.0,1.0
4,4,4.1,4.1.1,4,Afghanistan,2016,2016,25.0,16.0,166.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191757,4,4.c,4.c.1,894,Zambia,2012,2012,28.0,19.0,300.0,2.0,0.0,1.0,1.0
191758,4,4.c,4.c.1,894,Zambia,2012,2012,28.0,19.0,300.0,2.0,0.0,2.0,1.0
191759,4,4.c,4.c.1,894,Zambia,2017,2017,28.0,19.0,300.0,0.0,0.0,2.0,1.0
191760,4,4.c,4.c.1,894,Zambia,2017,2017,28.0,19.0,300.0,0.0,0.0,0.0,1.0


In [26]:
X_train = modeling_features(X_train)

In [27]:
X_test = modeling_features(X_test)

# Modeling

Lets do a quick grid search to determine the best regularization parameter. 

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
lasso_reg = Lasso() 
lasso_reg.get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'positive', 'precompute', 'random_state', 'selection', 'tol', 'warm_start'])

In [29]:

param_grid = [
    {'alpha': [.01, .1, .25, .5, .75, .99]}]





grid_search = GridSearchCV(lasso_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid=[{'alpha': [0.01, 0.1, 0.25, 0.5, 0.75, 0.99]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [30]:
grid_search.best_params_

{'alpha': 0.99}

In [31]:
lasso_reg = Lasso(alpha=0.1) 
lasso_reg.fit(X_train, y_train)

Lasso(alpha=0.1)

In [32]:
from sklearn.metrics import mean_squared_error
import numpy as np
#We make out final model
y_predictions = lasso_reg.predict(X_train)

final_mse = mean_squared_error(y_train, y_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

39.44143672452898

In [33]:
y_train.describe()

count    190313.000000
mean         45.361855
std          42.444991
min           0.000000
25%           1.012320
50%          42.592290
75%          89.781460
max        1632.065770
Name: Value, dtype: float64

Considering that the labels have an std of 42, our model doesn't perform too bad. 

In [37]:
lasso_reg.coef_.astype(int)

array([ 0,  1, -6,  3])

In [35]:
X_train.columns

Index(['GeoAreaCode', 'SeriesCode', 'Nature', 'Sex'], dtype='object')

To my suprise the model gives no weight to GeoAreaCode

Unfortunetly the modeling didn't give too much information. Series code can be valuable, it can indicate that our value increases by what we are measuring One-hot-encoding would be valuable for this but time does not permit it. 

Nature tells us what the data is. Whether it is country, country adjusted, or Estimated data. The data source causes our value to decrease. 

Lastly, Sex is the feature that makes most sense in this data. Many nations still suffer from Gender norms, and it shows in this data, the ordinal endoder set Male as 2, thus male adds the most to our value predictions

# Predicting on 2019

In [39]:
y_test_pred = lasso_reg.predict(X_test)
final_mse = mean_squared_error(y_test,y_test_pred)
final_rmse = np.sqrt(final_mse)
final_rmse

30.563854948566036

In [40]:
y_test.describe()

count    1302.000000
mean       38.183463
std        29.181939
min         0.000000
25%        13.174820
50%        33.684435
75%        57.078592
max       100.000000
Name: Value, dtype: float64

Our model generalized well when comparing to the standard deviation but is not the most reliable metric. Too much time was spent on cleaning and understaning the data.  